## Estudio de Convergencia: Calculos con PyGBe

### Lisozima 1hel sometida a un Campo Electrico Externo

#### Parametros PyGBe

	Use GPU                 : 1
	P                       : 6
	threshold               : 0.50
	theta                   : 0.50
	NCRIT                   : 500
	CUDA block size         : 128
	Gauss points per element: 4
	Gauss points near singlr: 19
	1D Gauss points per side: 9
	GMRES tolerance         : 1e-05
	GMRES max iterations    : 1000
	GMRES restart iteration : 200

### MSMS: 
#### Efield = 1e-12 y _k_ = 0.125

In [8]:
from numpy import *
from pylab import *
import os

#Calculo de Aproximacion Sol. Analitica por medio de Extrapolacion de Richardson:
#Esolv = array([-714.3092, -630.0634, -598.2773, -584.9413]) #kcal/mol con campo EF:5.5264e-5 sin trimesh
Esolv = array([-267.82944120,-258.3171181,-256.249229,-255.202304])
N=array([2,4,8,16])  #N° Elementos

r= N[3]/N[2]
p = log((Esolv[2] - Esolv[1])/(Esolv[3] - Esolv[2]))/log(r)
print("p_solv: ",p)
fsolvanalitico = Esolv[3] + ((Esolv[3] - Esolv[2])/((r**p) - 1))
print('E_solv_extrapolado:',fsolvanalitico,'Kcal/mol')

p_solv:  0.982000722965
E_solv_extrapolado: -254.128757971 Kcal/mol


In [15]:
#Calculo de Errores Relativos en Energia de Solvatacion:
PyGerr_solv = abs(Esolv-fsolvanalitico)/abs(fsolvanalitico)
print(PyGerr_solv*100)
asympt_solv = N[-1]*PyGerr_solv[-1]/N

figure(figsize=(3,2), dpi=80)
loglog(N, PyGerr_solv, c="k", marker="o", mfc="w", ms=3, ls="", lw=0.5, label="$E_{solv}$")
loglog(N, asympt_solv, c="k", marker="", mfc="w", ms=3, ls=":", lw=0.8, label=None)
text(1.6e4, 6e-2,"1/N",fontsize=4,rotation=-8)
text(0.5e5,0.5,"$E_{0}$ = 2.7632e-9 e/$\epsilon_{0}\mathrm{\AA}^{2}$",fontsize = 5)
ylabel('Error Esolv', fontsize=10)
xlabel('$N^{\circ}$ Elements',fontsize=10)
subplots_adjust(left=0.25, bottom=0.25, right=0.96, top=0.96)
axis([1,100,0.001, 1])
legend(loc='lower right', fontsize = '6')
savefig('Error_pepm1_Esolv_Ef2.7632e-9_k0.125_msms.pdf')
clf()

[ 5.39123684  1.64812521  0.83440813  0.42244177]


#### Efield = 1e-12 y _k_ = 0.086

In [3]:
#Calculo de Aproximacion Sol. Analitica por medio de Extrapolacion de Richardson:
Esolv = array([-701.2304, -616.6074, -584.5208, -571.0249]) #kcal/mol con campo EF:5.5264e-5 con trimesh
N=array([10328, 20012, 42844, 90260])  #N° Elementos

r= N[3]/N[2]
p = log((Esolv[2] - Esolv[1])/(Esolv[3] - Esolv[2]))/log(r)
print("p_solv: ", p)
fsolvanalitico = Esolv[3] + ((Esolv[3] - Esolv[2])/((r**p) - 1))
print('E_solv_extrapolado:',fsolvanalitico,'Kcal/mol')

p_solv:  1.16228574473
E_solv_extrapolado: -561.227564945 Kcal/mol


In [4]:
#Calculo de Errores Relativos en Energia de Solvatacion:
PyGerr_solv = abs(Esolv-fsolvanalitico)/abs(fsolvanalitico)
print(PyGerr_solv*100)
asympt_solv = N[-1]*PyGerr_solv[-1]/N

figure(figsize=(3,2), dpi=80)
loglog(N, PyGerr_solv, c="k", marker="o", mfc="w", ms=3, ls="", lw=0.5, label="$E_{solv}$")
loglog(N, asympt_solv, c="k", marker="", mfc="w", ms=3, ls=":", lw=0.8, label=None)
text(1.6e4, 6e-2,"1/N",fontsize=4,rotation=-8)
text(0.8e5,0.5,"$E_{0}$ = 1e-12 e/$\epsilon_{0}\mathrm{\AA}^{2}$",fontsize = 5)
ylabel('Error Esolv', fontsize=10)
xlabel('$N^{\circ}$ Elements',fontsize=10)
subplots_adjust(left=0.25, bottom=0.25, right=0.96, top=0.96)
axis([1000,1000000,0.001, 1])
legend(loc='lower right', fontsize = '6')
savefig('Error_1hel_Esolv_Ef1e-12_k0.086_nanoshaper.pdf')
clf()

[ 24.94582301   9.86762563   4.15040823   1.74569741]


### Lisozima 1hel cerca de Surf. Cargada C= -0.04 C/m2 sometida a un Campo Electrico Externo

#### Parametros PyGBe

	Use GPU                 : 1
	P                       : 4
	threshold               : 0.50
	theta                   : 0.50
	NCRIT                   : 500
	CUDA block size         : 128
	Gauss points per element: 2
	Gauss points near singlr: 19
	1D Gauss points per side: 9
	GMRES tolerance         : 1e-05
	GMRES max iterations    : 1000
	GMRES restart iteration : 200

### NanoShaper
#### Efield = 0.0 y _k_ = 0.086

In [1]:
from numpy import *
from pylab import *
import os

Esolv = array([-702.5513, -617.9361, -585.9047, -572.4428]) #kcal/mol sin campo
Esurf = array([14.5629, 14.5854, 14.6069, 14.6129]) #kcal/mol sin campo
NN=array([34328, 67952, 138844, 282020])#N° Elementos

r= NN[3]/NN[2]
r_surf = NN[3]/NN[2]
p_solv = log((Esolv[2] - Esolv[1])/(Esolv[3] - Esolv[2]))/log(r)
p_surf = log((Esurf[2] - Esurf[1])/(Esurf[3] - Esurf[2]))/log(r_surf)
print("p_solv: ", p_solv)
print("p_surf: ", p_surf)

fsolvanalitico = Esolv[3] + ((Esolv[3] - Esolv[2])/((r**p_solv) - 1))
fsurfanalitico = Esurf[2] + ((Esurf[2] - Esurf[1])/((r**p_surf) - 1))
print('E_solv_analitico:',fsolvanalitico,'Kcal/mol')
print('E_surf_analitico:',fsurfanalitico,'Kcal/mol')

p_solv:  1.22328560873
p_surf:  1.8010793894
E_solv_analitico: -562.683638385 Kcal/mol
E_surf_analitico: 14.6152225806 Kcal/mol


In [2]:
PyGerr_solv = abs(Esolv-fsolvanalitico)/abs(fsolvanalitico)
print(PyGerr_solv*100)
PyGerr_surf = abs(Esurf-fsurfanalitico)/abs(fsurfanalitico)
print(PyGerr_surf*100)
asympt_solv = NN[-1]*PyGerr_solv[-1]/NN
asympt_surf = NN[-3]*PyGerr_surf[-3]/NN
print(asympt_surf*100)

figure(figsize=(3,2), dpi=80)
loglog(NN, PyGerr_solv, c='k', marker='o', mfc='w', ms=3, ls='', lw=0.5, label="$E_{solv}$")
loglog(NN, asympt_solv, c='k', marker='', mfc='w', ms=3, ls=':', lw=0.8, label=None)
loglog(NN, PyGerr_surf, c='k', marker='^', mfc='w', ms=3, ls='', lw=0.5, label="$E_{surf}$")
loglog(NN, 100/NN, c='k', marker='', mfc='w', ms=3, ls=':', lw=0.8, label=None)
text(1.6e4, 6e-2,"1/N",fontsize=4,rotation=-8)
text(0.8e5,0.5,"$E_{0}$ = 1e-12 e/$\epsilon_{0}\mathrm{\AA}^{2}$",fontsize = 5)
ylabel('Error Esolv', fontsize=10)
xlabel('$N^{\circ}$ Elements',fontsize=10)
subplots_adjust(left=0.25, bottom=0.25, right=0.96, top=0.96)
axis([10000,1000000,0.00001, 1])
legend(loc='lower right', fontsize = '6')
savefig('Error_1hel-sensor_Esolv_Ef0.0_k0.086_nanoshaper.pdf')
clf()

[ 24.85724696   9.8194541    4.12684145   1.73439584]
[ 0.35800057  0.2040515   0.0569446   0.01589152]
[ 0.4039183   0.2040515   0.09986537  0.04916569]


#### Efield = 5.5264e-5 y _k_ = 0.086

In [1]:
#Esolv = array([-722.2449, -637.7406, -605.7249, -592.3012]) #kcal/mol con campo EF:5.5264e-5 #mal hecho move_protein.py
#Esurf = array([-18.7826, -18.7741, -18.7551, -18.7511]) #kcal/mol con campo EF:5.5264e-5 #y mal hechas mallas sensor
#NN=array([25128, 49012, 102044, 206034])#N° Elementos

#Esolv = array([-719.0277, -634.4544, -602.4989, -589.0468])
#Esurf = array([-19.4511, -19.4353, -19.4174, -19.4118])
#NN=array([34328, 67952, 138844, 282020])

#r= NN[3]/NN[2]
#r_surf = NN[3]/NN[2]
#p_solv = log((Esolv[2] - Esolv[1])/(Esolv[3] - Esolv[2]))/log(r)
#p_surf = log((Esurf[2] - Esurf[1])/(Esurf[3] - Esurf[2]))/log(r_surf)
#print("p_solv: ", p_solv)
#print("p_surf: ", p_surf)
#fsolvanalitico = Esolv[3] + ((Esolv[3] - Esolv[2])/((r**p_solv) - 1))
#fsurfanalitico = Esurf[2] + ((Esurf[2] - Esurf[1])/((r**p_surf) - 1))
#print('E_solv_analitico:',fsolvanalitico,'Kcal/mol')
#print('E_surf_analitico:',fsurfanalitico,'Kcal/mol')

In [2]:
#PyGerr_solv = abs(Esolv-fsolvanalitico)/abs(fsolvanalitico)
#print(PyGerr_solv*100)
#PyGerr_surf = abs(Esurf-fsurfanalitico)/abs(fsurfanalitico)
#print(PyGerr_surf*100)
#asympt_solv = NN[-1]*PyGerr_solv[-1]/NN
#asympt_surf = NN[-2]*PyGerr_surf[-2]/NN

#figure(figsize=(3,2), dpi=80)
#loglog(NN, PyGerr_solv, c='k', marker='o', mfc='w', ms=3, ls='', lw=0.5, label="$E_{solv}$")
#loglog(NN, asympt_solv, c='k', marker='', mfc='w', ms=3, ls=':', lw=0.8, label=None)
#loglog(NN, PyGerr_surf, c='k', marker='^', mfc='w', ms=3, ls='', lw=0.5, label="$E_{surf}$")
#loglog(NN, asympt_surf, c='k', marker='', mfc='w', ms=3, ls=':', lw=0.8, label=None)
#ylabel('error relativo', fontsize=10)
#xlabel('N mesh',fontsize=10)
#text(1.6e4, 6e-2,"1/N",fontsize=4,rotation=-8)
#text(0.8e5,0.3,"$E_{0}$ = 5.5264e-5 e/$\epsilon_{0}\mathrm{\AA}^{2}$",fontsize = 5)
#subplots_adjust(left=0.25, bottom=0.25, right=0.92, top=0.92)
#axis([10000,1000000,0.000001, 1])
#legend(loc='lower right', fontsize = '6')
#savefig('Error_1hel-sensor_Esolv_Ef5.5264e-5_k0.086_nanoshaper.pdf')
#clf()